In [36]:
import pandas as pd
# 数据
data=pd.read_csv('../Assignment1//dataset_mood_smartphone.csv')
data.set_index(data.columns[0],inplace=True)
data.reset_index(inplace=True)
data.index += 1
data.head(5)

,Unnamed: 0,id,time,variable,value
1,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
2,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
3,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
4,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
5,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [37]:
import numpy as np
print(data['value'].iloc[5708])
print(data['value'].iloc[5708] is np.NaN)

nan
False


In [38]:
import re

# 记录不符合规定的行索引

data_schema = pd.read_json("../Assignment1//threshold.json")


# 遍历每一行
def clean_raw(data, data_schema):
    invalid_rows = []
    error_types = []
    for index, row in data.iterrows():
        # 检查时间是否符合规定
        if not re.match(data_schema["time"]["pattern"], row["time"]):
            invalid_rows.append(index + 1)
            error_types.append("wrong_time")
            continue

        # 检查变量是否符合规定
        variable = row["variable"]
        value = row["value"]
        if variable in data_schema["variable"]:
            var_schema = data_schema["variable"][variable]
            if var_schema["range"] is not None:
                min_value, max_value = var_schema["range"]
                if not min_value <= value <= max_value:
                    if pd.isnull(value):
                        invalid_rows.append(index + 1)
                        error_types.append("missing_value")
                    else:
                        invalid_rows.append(index + 1)
                        error_types.append("out_of_range")
        else:
            invalid_rows.append(index + 1)
            error_types.append("missing_variable")

    print(len(invalid_rows))
    print("invalid row id:", invalid_rows)
    print("error types:", error_types)

In [39]:
clean_raw(data, data_schema)

206
invalid row id: [5710, 5732, 5774, 5798, 5837, 6326, 6380, 6435, 6669, 6794, 7038, 7257, 7263, 7321, 7349, 7451, 8194, 8203, 8351, 8358, 8363, 8384, 8405, 8462, 8468, 8644, 9333, 9391, 9395, 9400, 9444, 9479, 9504, 9520, 9647, 9920, 10190, 10242, 10249, 10263, 10284, 10293, 10294, 10330, 10335, 11257, 11301, 11353, 11380, 11381, 11383, 11416, 11417, 11420, 11480, 11481, 11489, 11498, 11969, 12023, 12068, 12078, 12312, 12325, 12437, 12681, 12775, 12900, 12925, 13038, 13048, 13052, 13053, 13062, 13180, 13184, 13188, 13204, 13750, 13800, 13821, 13824, 13827, 13844, 13846, 13854, 13861, 13888, 13898, 13929, 13946, 13960, 13962, 13966, 13975, 13978, 13993, 13994, 14001, 14004, 14009, 14020, 14023, 14027, 14029, 14036, 14039, 14045, 14048, 14055, 14056, 14057, 14064, 14071, 14072, 14086, 14090, 14098, 14102, 14105, 14106, 14107, 14111, 14114, 14287, 14313, 14314, 14316, 14325, 14330, 14331, 14337, 14340, 14936, 14940, 14973, 14976, 14983, 14985, 15026, 15031, 15033, 15034, 15038, 15040, 

KNN
------------------------------------------------------------------------------------------------------------------------------

# Convert 'time' column to datetime format
data['time'] = pd.to_datetime(data['time'])

# Checking the unique variables in the dataset
unique_variables = data['variable'].unique()

unique_variables


from sklearn.impute import KNNImputer

# Filtering the data for 'mood' variable
mood_data = data[data['variable'] == 'mood'].copy()

# Sorting the data by time to maintain time series order
mood_data.sort_values(by='time', inplace=True)

# We need to reset the index because KNNImputer relies on numerical indices to find nearest neighbors
mood_data.reset_index(drop=True, inplace=True)

# Selecting columns for imputation (using 'value' column only here as example)
values_for_imputation = mood_data[['value']]

# Setting up the KNN imputer, choosing 5 neighbors for simplicity
imputer = KNNImputer(n_neighbors=5)

# Performing the imputation
mood_data['value_imputed'] = imputer.fit_transform(values_for_imputation)

# Showing the original and imputed values to compare
mood_data[mood_data['value'].isnull()][['time', 'value', 'value_imputed']]


# Counting missing values by variable type
missing_by_variable = data.groupby('variable')['value'].apply(lambda x: x.isnull().sum())
missing_by_variable_sorted = missing_by_variable.sort_values(ascending=False)

missing_by_variable_sorted

# Filtering the data for 'circumplex.valence' variable
valence_data = data[data['variable'] == 'circumplex.arousal'].copy()

# Sorting the data by time to maintain time series order
valence_data.sort_values(by='time', inplace=True)

# Reset the index because KNNImputer relies on numerical indices to find nearest neighbors
valence_data.reset_index(drop=True, inplace=True)

# Selecting columns for imputation (using 'value' column only here as example)
valence_values_for_imputation = valence_data[['value']]

# Setting up the KNN imputer, choosing 5 neighbors for simplicity
valence_imputer = KNNImputer(n_neighbors=5)

# Performing the imputation
valence_data['value_imputed'] = valence_imputer.fit_transform(valence_values_for_imputation)

# Showing the original and imputed values to compare
valence_data[valence_data['value'].isnull()][['time', 'value', 'value_imputed']].head()


随机森林

In [40]:
# Step 1: Remove the unnecessary 'Unnamed: 0' column
data = data.drop(columns=['Unnamed: 0'])

# Step 2: Convert 'time' column to datetime format
data['time'] = pd.to_datetime(data['time'])

# Step 3: Analyze missing data by 'variable' type
missing_data_analysis = data.groupby('variable').apply(lambda x: x['value'].isnull().sum())

# Display the missing data analysis by variable
missing_data_analysis


/tmp/ipykernel_43512/1818056308.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_data_analysis = data.groupby('variable').apply(lambda x: x['value'].isnull().sum())


variable
activity                  0
appCat.builtin            0
appCat.communication      0
appCat.entertainment      0
appCat.finance            0
appCat.game               0
appCat.office             0
appCat.other              0
appCat.social             0
appCat.travel             0
appCat.unknown            0
appCat.utilities          0
appCat.weather            0
call                      0
circumplex.arousal       46
circumplex.valence      156
mood                      0
screen                    0
sms                       0
dtype: int64

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Label encoding for categorical data
label_encoder_id = LabelEncoder()
data['id_encoded'] = label_encoder_id.fit_transform(data['id'])

label_encoder_variable = LabelEncoder()
data['variable_encoded'] = label_encoder_variable.fit_transform(data['variable'])

# Function to prepare data for filling missing values with RandomForest
def prepare_data_for_rf(variable_name):
    # Selecting rows related to the variable
    variable_data = data[data['variable'] == variable_name]

    # Separating into data with and without missing values
    known_data = variable_data[variable_data['value'].notna()]
    unknown_data = variable_data[variable_data['value'].isna()]

    # Selecting features and target
    features = known_data[['id_encoded', 'variable_encoded', 'time']]
    target = known_data['value']

    # One-hot encoding for time features
    features = pd.get_dummies(features, columns=['time'])

    # Splitting the data for training
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, unknown_data, features.columns

# Prepare data for 'circumplex.arousal'
X_train_arousal, X_test_arousal, y_train_arousal, y_test_arousal, unknown_data_arousal, feature_cols_arousal = prepare_data_for_rf('circumplex.arousal')

# Initialize and train the RandomForest model
model_arousal = RandomForestRegressor(n_estimators=100, random_state=42)
model_arousal.fit(X_train_arousal, y_train_arousal)

# Predict the missing values
unknown_data_arousal_features = pd.get_dummies(unknown_data_arousal[['id_encoded', 'variable_encoded', 'time']], columns=['time'])
unknown_data_arousal_features = unknown_data_arousal_features.reindex(columns=feature_cols_arousal, fill_value=0)
predicted_values_arousal = model_arousal.predict(unknown_data_arousal_features)

# Display predictions
print(predicted_values_arousal)

# Assuming 'predicted_values_arousal' is a numpy array or a Pandas Series
unknown_data_arousal['value'] = predicted_values_arousal

# Fill the predicted values back to the original DataFrame
data.loc[unknown_data_arousal.index, 'value'] = unknown_data_arousal['value']


[-0.67 -0.56 -0.43 -0.62 -0.65 -0.55 -0.17 -0.4  -0.98 -0.3  -0.98 -0.01
  0.    0.01  0.    0.    1.    1.    0.67  0.82  0.59  1.22  0.82  0.82
  0.82  1.    0.    0.01  0.14  0.16  0.14  0.14  0.14  0.14 -0.01  0.06
 -0.28  0.98 -0.29 -0.14 -0.28  0.48 -0.3  -0.28 -0.28 -0.36]


In [42]:
# Prepare data for 'circumplex.valence'
(
    X_train_valence,
    X_test_valence,
    y_train_valence,
    y_test_valence,
    unknown_data_builtin,
    feature_cols_builtin,
) = prepare_data_for_rf("circumplex.valence")

# Initialize and train the RandomForest model for 'circumplex.valence'
model_valence = RandomForestRegressor(n_estimators=100, random_state=42)
model_valence.fit(X_train_valence, y_train_valence)

# Predict the missing values for 'circumplex.valence'
unknown_data_valence_features = pd.get_dummies(
    unknown_data_builtin[["id_encoded", "variable_encoded", "time"]], columns=["time"]
)
unknown_data_valence_features = unknown_data_valence_features.reindex(
    columns=feature_cols_builtin, fill_value=0
)
predicted_values_valence = model_valence.predict(unknown_data_valence_features)

# Display predictions for 'circumplex.valence'
print(predicted_values_valence)
unknown_data_builtin["value"] = predicted_values_valence
data.loc[unknown_data_builtin.index, "value"] = unknown_data_builtin["value"]

[ 1.    0.8   1.    0.99  0.9   1.    1.    0.82  1.    1.    1.01  1.
  1.    1.    1.    0.97  1.    1.    1.    0.82  1.    0.95  0.92  1.
  1.    1.    0.52  1.    1.    0.84  1.    1.    1.    1.16  1.    1.
  0.98  1.    1.    1.    0.92  1.    0.41  0.79  1.    1.    0.76  1.
  1.    1.    1.    0.45  0.92  1.    1.    1.    1.    1.    0.99  1.
  0.94 -0.31  0.99  1.    0.8   0.84  1.    1.    1.    0.5  -0.12  1.
  0.96  1.    0.41  0.37  0.28  1.    1.    1.    0.47  0.96  1.    1.
  0.87  1.    1.    1.    1.    1.    0.99  0.98  0.9   1.    1.    1.
  1.    1.    1.    1.    1.    1.    0.98  1.    1.    1.    1.    1.
  1.    1.    1.    1.    1.    1.    0.46  1.    1.    1.    1.    1.
  1.    1.    1.    0.86  1.53  1.    0.94  0.59  1.    0.77  1.    0.46
  1.    0.51  0.44  1.    0.45  1.    1.    1.    1.    1.    1.    1.
  1.    0.97  0.99  0.97  0.27  0.97  0.97  0.97  0.97  0.97  0.97  0.97]


In [43]:
print(data.head(5))


        id                time variable  value  id_encoded  variable_encoded
1  AS14.01 2014-02-26 13:00:00     mood    6.0           0                16
2  AS14.01 2014-02-26 15:00:00     mood    6.0           0                16
3  AS14.01 2014-02-26 18:00:00     mood    6.0           0                16
4  AS14.01 2014-02-26 21:00:00     mood    7.0           0                16
5  AS14.01 2014-02-27 09:00:00     mood    6.0           0                16


In [44]:
# List of invalid row ids
invalid_row_ids = [150042, 159974, 162156, 309807]

# Subtract 1 from all ids
invalid_row_ids = [id-1 for id in invalid_row_ids]

# Select these rows
invalid_rows = data[data.index.isin(invalid_row_ids)]

# Print the 'value' column of these rows
print(invalid_rows['value'])
data.loc[invalid_row_ids, 'value'] = np.nan

150041      -44.689
159973   -82798.871
162155       -1.218
309806       -0.011
Name: value, dtype: float64


In [45]:
data = data.dropna(subset=['value'])
# Convert 'time' column to string
data['time'] = data['time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
# Remove the last 3 digits (microseconds to milliseconds)
data['time'] = data['time'].str[:-3]
data = data.drop(['id_encoded', 'variable_encoded'], axis=1)

In [46]:
clean_raw(data,data_schema)

0
invalid row id: []
error types: []


In [47]:
data.to_csv('cleaned_dataset_mood_smartphone.csv')